In [ ]:
import sqlite3 # Importando a biblioteca necessária
import pandas as pd

# Carrega a base. Assumindo que o arquivo do M25 é TB_VENDAS_TAREFA (1).csv
df_vendas = pd.read_csv("TB_VENDAS_TAREFA (1).csv", delimiter=';') 

# Conecta ao BD em memória
conn = sqlite3.connect(':memory:')
df_vendas.to_sql('tb_vendas', conn, index=False, if_exists='replace') # Carrega a tabela

# Função para executar queries e retornar um DataFrame
def run_query(query): 
    return pd.read_sql_query(query, conn)

    query = """
SELECT 
    PRODUTO, 
    UNIDADES 
FROM 
    tb_vendas 
WHERE 
    PRODUTO = 'CAMISETA'
"""
result_df = run_query(query)
print(result_df)

query = """
SELECT 
    * FROM 
    tb_vendas 
WHERE 
    UNIDADES >= 3
"""
result_df = run_query(query)
print(result_df)

query = """
SELECT 
    PRODUTO, 
    UNIDADES 
FROM 
    tb_vendas 
WHERE 
    CAST(REPLACE(VALOR_UNID, ',', '.') AS REAL) >= 50
"""
result_df = run_query(query)
print(result_df)

# Explicação da Query 3: Filtragem Numérica

Nesta consulta, utilizamos a cláusula `WHERE` para aplicar a condição de filtro.

O ponto crucial aqui é o **tratamento do tipo de dado**:

1.  **`REPLACE(VALOR_UNID, ',', '.')`**: O campo `VALOR_UNID` foi importado com vírgula (`,`) como separador decimal. O SQL Lite espera o ponto (`.`). Usamos `REPLACE` para trocar todas as vírgulas por pontos.
2.  **`CAST(... AS REAL)`**: Em seguida, usamos `CAST` para converter a *string* resultante em um tipo numérico (`REAL`, que é o decimal), permitindo a comparação correta com `>= 50`.

A consulta retorna todos os produtos e unidades com valor de unidade igual ou superior a 50,00.

query = """
SELECT 
    ID_COMPRA, 
    CAST(REPLACE(VALOR_UNID, ',', '.') AS REAL) * UNIDADES AS VALOR_TOTAL_GASTO 
FROM 
    tb_vendas 
ORDER BY 
    VALOR_TOTAL_GASTO DESC 
LIMIT 5
"""
result_df = run_query(query)
print(result_df)

# Explicação da Query 4: TOP 5 e Ordenação

Esta consulta resolve o desafio de encontrar as transações de maior valor em três etapas principais:

1.  **Cálculo (SELECT)**: Calculamos o `VALOR_TOTAL_GASTO` multiplicando `VALOR_UNID` por `UNIDADES`, mantendo o tratamento de vírgula/ponto para o cálculo numérico.
2.  **Ordenação (ORDER BY)**: Usamos `ORDER BY VALOR_TOTAL_GASTO DESC` para classificar todos os resultados do maior valor total para o menor.
3.  **Restrição (LIMIT)**: A cláusula `LIMIT 5` restringe o resultado final apenas às 5 primeiras linhas, que correspondem às compras com o maior valor total gasto.

query = """
SELECT 
    PRODUTO, 
    AVG(CAST(REPLACE(VALOR_UNID, ',', '.') AS REAL)) AS MEDIA_PRECO_UNIDADE 
FROM 
    tb_vendas 
GROUP BY 
    PRODUTO 
ORDER BY 
    MEDIA_PRECO_UNIDADE DESC
"""
result_df = run_query(query)
print(result_df)

# Explicação da Query 5: Agregação e Ordenação

Esta consulta utiliza funções de agregação para resumir os dados por categoria de produto:

1.  **Agregação (`AVG` e `GROUP BY`)**: Usamos a função **`AVG()`** para calcular a média do `VALOR_UNID` (com o tratamento de dados necessário) para cada produto. A cláusula **`GROUP BY PRODUTO`** é obrigatória aqui, pois ela instrui o SQL a calcular a média separadamente para cada item único na coluna `PRODUTO`.
2.  **Ordenação (`ORDER BY`)**: O resultado final é ordenado usando **`ORDER BY MEDIA_PRECO_UNIDADE DESC`**, o que exibe os produtos com a maior média de preço por unidade primeiro.

query = """
SELECT 
    ID_CLIENTE, 
    SUM(UNIDADES) AS TOTAL_UNIDADES_COMPRADAS, 
    COUNT(ID_COMPRA) AS NUMERO_DE_COMPRAS 
FROM 
    tb_vendas 
GROUP BY 
    ID_CLIENTE
"""
result_df = run_query(query)
print(result_df)

# Explicação da Query 6: KPIs de Clientes

Esta consulta serve para obter KPIs (Key Performance Indicators) do comportamento de compra de cada cliente:

1.  **Agrupamento (`GROUP BY`)**: O comando **`GROUP BY ID_CLIENTE`** garante que as agregações sejam calculadas no nível de cada cliente.
2.  **Soma (`SUM`)**: Usamos **`SUM(UNIDADES)`** para calcular o volume total de itens comprados por cada cliente.
3.  **Contagem (`COUNT`)**: Usamos **`COUNT(ID_COMPRA)`** para contar o número de linhas (registros de compra) associadas a cada cliente, fornecendo o número total de compras realizadas.